# 🎯 Summarising and Presenting Data

**DS105W W05 LECTURE – Data for Data Science (Winter Term 2025/2026)**

<div style="font-family: system-ui; padding: 20px 30px 20px 20px; background-color: #FFFFFF; color: #212121; border-left: 8px solid #ED9255; border-radius: 8px; box-shadow: 0 4px 12px rgba(0, 0, 0, 0.1);max-width:600px;">

**Lecture Demonstration Notebook**

- 📅 Date: 19 February 2026
- 👤 Instructor: Dr Jon Cardoso-Silva
- 🎯 Purpose: Master advanced pandas transformations and present data with Styler

<span style="display:block;line-height:1.15em;color:#666666;font-size:0.9em;">

🥅 **Learning Goals**

 i) Write custom functions for complex logic and apply them to entire datasets,
 ii) Convert timestamps and extract date components for temporal analysis,
 iii) Use .groupby() to summarise data and reveal patterns,
 iv) Format and style summary tables for presentation using pandas Styler,
 v) Write narrative captions that communicate findings.

</span>

</div>


## What You'll Learn Today

Last week you struggled with nested `np.where()` and boolean columns in the W04 Lab when classifying weather. Today, you'll learn a much cleaner approach through **custom functions** and how to **summarise temporal data** to reveal insights. Finally, you'll learn to **present your summary tables** using pandas Styler.

**Today's path:** Custom functions → DateTime operations → GroupBy aggregations → Presentation with Styler

**Why this matters:** These skills directly support your ✍️ [Mini-Project 1](https://lse-dsi.github.io/DS105/2025-2026/winter-term/summative/mini-project-1.html) work.


💭 **Personal Reflection:**

As we go through today's lecture, use this space to write your own notes, questions, or observations.

- [*Write your notes here*]


⚙️ **Importing libraries**

We'll use several libraries today:


In [1]:
import json

import numpy as np
import pandas as pd

from datetime import datetime

print("✅ Libraries loaded successfully!")

✅ Libraries loaded successfully!


📖 **Learn more:**
- [Pandas .apply() Method](https://pandas.pydata.org/docs/reference/api/pandas.Series.apply.html) (Official Docs)
- [Pandas .groupby() Guide](https://pandas.pydata.org/docs/user_guide/groupby.html) (Official Docs)
- [Pandas Styler Guide](https://pandas.pydata.org/docs/user_guide/style.html) (Official Docs)


## Setup: Load Extended Weather Data

We'll use 20 years of London weather data (2006-2025) with both temperature and rainfall.


In [2]:
with open('data/london_weather_2006_2025.json', 'r') as f:
    weather_data = json.load(f)

dates = weather_data['daily']['time']
temps = weather_data['daily']['temperature_2m_max']
rainfall = weather_data['daily']['rain_sum']

print(f"✅ Loaded {len(dates)} days of data")
print(f"📅 From {dates[0]} to {dates[-1]}")
print(f"🌡️  Temperature range: {min(temps):.1f}°C to {max(temps):.1f}°C")
print(f"🌧️  Rainfall range: {min(rainfall):.1f}mm to {max(rainfall):.1f}mm")

✅ Loaded 7305 days of data
📅 From 2006-01-01 to 2025-12-31
🌡️  Temperature range: -1.5°C to 37.9°C
🌧️  Rainfall range: 0.0mm to 51.6mm


In [3]:
print("First 10 days:")
print("-" * 70)
for i in range(10):
    print(f"{dates[i]}: {temps[i]:>5.1f}°C, {rainfall[i]:>5.2f}mm")

First 10 days:
----------------------------------------------------------------------
2006-01-01:   6.4°C,  1.30mm
2006-01-02:   5.9°C,  0.00mm
2006-01-03:   7.7°C,  3.10mm
2006-01-04:   7.0°C,  0.00mm
2006-01-05:   2.7°C,  0.00mm
2006-01-06:   4.7°C,  0.00mm
2006-01-07:   3.8°C,  0.50mm
2006-01-08:   4.2°C,  4.60mm
2006-01-09:   6.7°C,  0.50mm
2006-01-10:   8.0°C,  0.00mm


💭 **Personal Reflection:**

How is this dataset different from the W04 Lab data? What new possibilities does rainfall add?

- [*Write your notes here*]


## Section 1: From Loops to Functions

Remember the W04 Lab struggle with nested `np.where()` for weather classification?


In [4]:
weather_df = pd.DataFrame({
    'date': dates,
    'temp': temps,
    'rain_mm': rainfall
})

weather_df.head(10)

,date,temp,rain_mm
0,2006-01-01,6.4,1.3
1,2006-01-02,5.9,0.0
2,2006-01-03,7.7,3.1
3,2006-01-04,7.0,0.0
4,2006-01-05,2.7,0.0
5,2006-01-06,4.7,0.0
6,2006-01-07,3.8,0.5
7,2006-01-08,4.2,4.6
8,2006-01-09,6.7,0.5
9,2006-01-10,8.0,0.0


In [5]:
print(f"Shape: {weather_df.shape}")
print(f"Columns: {weather_df.columns.tolist()}")

Shape: (7305, 3)
Columns: ['date', 'temp', 'rain_mm']


Remember the W04 Lab challenge? Classify weather based on temperature AND rainfall:

```markdown
Hot & Dry   (temp > 25°C AND rain < 1mm)
Hot & Wet   (temp > 25°C AND rain >= 1mm)
Mild & Dry  (temp 20-25°C AND rain < 1mm)
Mild & Wet  (temp 20-25°C AND rain >= 1mm)
Cool        (temp < 20°C, any rain)
```

With nested `np.where()`, this becomes unreadable. 😅


### Custom Functions: Making Logic Readable

A function is a reusable block of code that takes inputs and produces an output.

Think of it as one iteration of your loop, made reusable.


### Writing Your First Function


In [6]:
def is_hot(temp):
    """
    Check if temperature is considered hot.
    
    Parameters
    ----------
    temp : float
        Temperature in Celsius
    
    Returns
    -------
    bool
        True if temp >= 25°C, False otherwise
    """
    if temp >= 25:
        return True
    else:
        return False

print("Testing the function:")
print(f"30°C: {is_hot(30)}")
print(f"20°C: {is_hot(20)}")
print(f"25°C: {is_hot(25)}")  # Boundary case

Testing the function:
30°C: True
20°C: False
25°C: True


Anatomy of a function:

- `def` keyword starts the function definition
- Function name should be descriptive
- Docstring explains what it does
- Parameters are the inputs
- `return` statement produces the output


<div style="background-color: #fcfcfc; width:80%; margin-left: 1em; color: #212121; padding: 1em; border-radius: 0.5em; border: 1px solid #4caf50; border-left: 5px solid #4caf50;">

Testing your function on a few examples first catches bugs before you apply it to thousands of rows, and it is easier to debug when you have a small set of inputs to trace through.

</div>

💭 **Personal Reflection:**

Why do we test the function on single values before applying it to all data?

- [*Write your notes here*]


### Test Edge Cases


In [7]:
print("Testing edge cases:")
print(f"Exactly 25°C: {is_hot(25)}")   # Should be True
print(f"Exactly 20°C: {is_hot(20)}")   # Should be False
print(f"24.9°C: {is_hot(24.9)}")       # Should be False

Testing edge cases:
Exactly 25°C: True
Exactly 20°C: False
24.9°C: False


### Series.apply(): Iteration Abstraction


In [8]:
weather_df['is_hot'] = weather_df['temp'].apply(is_hot)

weather_df.head(15)

,date,temp,rain_mm,is_hot
0,2006-01-01,6.4,1.3,False
1,2006-01-02,5.9,0.0,False
2,2006-01-03,7.7,3.1,False
3,2006-01-04,7.0,0.0,False
4,2006-01-05,2.7,0.0,False
5,2006-01-06,4.7,0.0,False
6,2006-01-07,3.8,0.5,False
7,2006-01-08,4.2,4.6,False
8,2006-01-09,6.7,0.5,False
9,2006-01-10,8.0,0.0,False


In [9]:
print(f"Total hot days: {weather_df['is_hot'].sum()}")

Total hot days: 302


Pandas called `is_hot()` for each temperature value, just like a `for` loop would. The difference: your logic is now extracted, testable, and reusable.


### One-liners with lambda

For quick, inline logic, `lambda` lets you write a one-liner function.
Use sparingly; prefer `def` when logic grows.

In [10]:
weather_df['is_hot_lambda'] = weather_df['temp'].apply(lambda t: t >= 25)

### Complex Weather Classification Function


In [11]:
def classify_weather(row):
    """
    Classify weather based on temperature AND rainfall.
    
    Parameters
    ----------
    row : pd.Series
        A DataFrame row with 'temp' and 'rain_mm' columns
    
    Returns
    -------
    str
        Weather classification
    """
    temp = row['temp']
    rain = row['rain_mm']
    
    if temp >= 25 and rain < 1:
        return "Hot & Dry"
    elif temp >= 25:
        return "Hot & Wet"
    elif temp >= 20 and rain < 1:
        return "Mild & Dry"
    elif temp >= 20:
        return "Mild & Wet"
    else:
        return "Cool"

test_row = weather_df.iloc[0]
print(test_row)
print(f"Test: {test_row['temp']}°C, {test_row['rain_mm']:.2f}mm")
print(f"Result: {classify_weather(test_row)}")
display(weather_df.head(1))


date             2006-01-01
temp                    6.4
rain_mm                 1.3
is_hot                False
is_hot_lambda         False
Name: 0, dtype: object
Test: 6.4°C, 1.30mm
Result: Cool


,date,temp,rain_mm,is_hot,is_hot_lambda
0,2006-01-01,6.4,1.3,False,False


In [12]:
weather_df['weather_type'] = weather_df.apply(classify_weather, axis=1)

weather_df[['date', 'temp', 'rain_mm', 'weather_type']].head(20)

,date,temp,rain_mm,weather_type
0,2006-01-01,6.4,1.3,Cool
1,2006-01-02,5.9,0.0,Cool
2,2006-01-03,7.7,3.1,Cool
3,2006-01-04,7.0,0.0,Cool
4,2006-01-05,2.7,0.0,Cool
5,2006-01-06,4.7,0.0,Cool
6,2006-01-07,3.8,0.5,Cool
7,2006-01-08,4.2,4.6,Cool
8,2006-01-09,6.7,0.5,Cool
9,2006-01-10,8.0,0.0,Cool


In [13]:
print("Weather type distribution:")
print(weather_df['weather_type'].value_counts())

Weather type distribution:
weather_type
Cool          5767
Mild & Dry     916
Mild & Wet     320
Hot & Dry      265
Hot & Wet       37
Name: count, dtype: int64


<div style="background-color: #fcfcfc; width:80%; margin-left: 1em; color: #212121; padding: 1em; border-radius: 0.5em; border: 1px solid #4caf50; border-left: 5px solid #4caf50;">

**Compare this to nested `np.where()`:** Which is easier to read? Which would be easier to debug if there was a mistake?

</div>


💭 **Personal Reflection:**

How would you modify the function to add a "Very Hot" category for temperatures above 30°C?

- [*Write your notes here*]


## Section 2: Temporal Data

To answer questions like "Is London's air getting better or worse?" you need to work with dates and times.


### Converting Strings to DateTime


In [14]:
weather_df['date_dt'] = pd.to_datetime(weather_df['date'])

weather_df[['date', 'date_dt']].head(10)

,date,date_dt
0,2006-01-01,2006-01-01
1,2006-01-02,2006-01-02
2,2006-01-03,2006-01-03
3,2006-01-04,2006-01-04
4,2006-01-05,2006-01-05
5,2006-01-06,2006-01-06
6,2006-01-07,2006-01-07
7,2006-01-08,2006-01-08
8,2006-01-09,2006-01-09
9,2006-01-10,2006-01-10


In [15]:
print(f"Original date column type: {weather_df['date'].dtype}")
print(f"New datetime column type: {weather_df['date_dt'].dtype}")

Original date column type: str
New datetime column type: datetime64[us]


### The .dt Accessor


In [16]:
weather_df['year'] = weather_df['date_dt'].dt.year
weather_df['month'] = weather_df['date_dt'].dt.month
weather_df['day'] = weather_df['date_dt'].dt.day
weather_df['dayofweek'] = weather_df['date_dt'].dt.dayofweek  # Monday=0

weather_df[['date', 'year', 'month', 'day', 'dayofweek']].head(10)

,date,year,month,day,dayofweek
0,2006-01-01,2006,1,1,6
1,2006-01-02,2006,1,2,0
2,2006-01-03,2006,1,3,1
3,2006-01-04,2006,1,4,2
4,2006-01-05,2006,1,5,3
5,2006-01-06,2006,1,6,4
6,2006-01-07,2006,1,7,5
7,2006-01-08,2006,1,8,6
8,2006-01-09,2006,1,9,0
9,2006-01-10,2006,1,10,1


Think of `.dt` as datetime-specific operations, similar to `.str` for strings.


💭 **Personal Reflection:**

What other datetime components might be useful for weather analysis?

- [*Write your notes here*]


## Live Exercise:


In [17]:
weather_df['miserable'] = (weather_df['is_hot'] == True) | ((weather_df['rain_mm'] > 1) & (weather_df['temp'] < 10))
print(weather_df['miserable'].sum())

834


In [18]:
weather_df['temp'].apply(is_hot)

0       False
1       False
2       False
3       False
4       False
        ...  
7300    False
7301    False
7302    False
7303    False
7304    False
Name: temp, Length: 7305, dtype: bool

## Section 3: Split -> Apply -> Combine

**Split-apply-combine** mental model:

1. **Split** data into groups (by year, by category)
2. **Apply** a function to each group
3. **Combine** results back together


### The groupby() method

Pandas provides `groupby()` to summarise data by groups:

```python
weather_df.groupby('year')['temp'].mean()
```

This splits by `year`, computes the mean of `temp` per year, then combines the results.


### Basic GroupBy


In [19]:
yearly_temps = weather_df.groupby('year')['temp'].mean()
print("Average temperature by year:")
print(yearly_temps.head(10))

Average temperature by year:
year
2006    14.675342
2007    14.390685
2008    13.793443
2009    13.921370
2010    12.749589
2011    14.774795
2012    13.512295
2013    13.309589
2014    14.978356
2015    14.336986
Name: temp, dtype: float64


In [20]:
hot_days_by_year = weather_df.groupby('year')['is_hot'].sum()
print("\nHot days by year:")
print(hot_days_by_year.head(10))


Hot days by year:
year
2006    24
2007     1
2008     3
2009     6
2010    11
2011     8
2012     6
2013    18
2014     9
2015     4
Name: is_hot, dtype: int64


### Multiple Aggregations


In [21]:
yearly_stats = weather_df.groupby('year')['temp'].agg(['mean', 'max', 'min', 'std'])
print("Temperature statistics by year:")
print(yearly_stats.head(10))

Temperature statistics by year:
           mean   max  min       std
year                                
2006  14.675342  31.5  0.8  6.764682
2007  14.390685  26.4  1.6  5.042399
2008  13.793443  26.6  2.0  5.497393
2009  13.921370  26.5 -0.1  6.040721
2010  12.749589  28.0 -1.5  7.180979
2011  14.774795  28.6  1.5  5.505723
2012  13.512295  28.1 -0.5  5.665366
2013  13.309589  30.4 -0.9  6.817386
2014  14.978356  28.5  4.0  5.285908
2015  14.336986  32.5  2.4  5.116642


In [22]:
yearly_weather = weather_df.groupby('year')[['temp', 'rain_mm']].mean()
print("\nAverage temperature and rainfall by year:")
print(yearly_weather.head(10))


Average temperature and rainfall by year:
           temp   rain_mm
year                     
2006  14.675342  1.709863
2007  14.390685  1.747945
2008  13.793443  1.821311
2009  13.921370  1.613973
2010  12.749589  1.489863
2011  14.774795  1.289315
2012  13.512295  2.314208
2013  13.309589  1.647123
2014  14.978356  2.142192
2015  14.336986  1.732329


### Method Chaining for Readability

Splitting long transformations across lines improves clarity:

```python
yearly_temps_chain = (
    weather_df
    .groupby('year')['temp']
    .mean()
    .reset_index()
)
```

Hot days by month using chaining:

```python
hot_days_per_month = (
    weather_df.loc[weather_df['is_hot']]
    .groupby('month')
    .size()
)
```


### You can group by multiple columns

Average temperature by year and month:

```python
yearly_monthly_temps = (
    weather_df
    .groupby(['year', 'month'])['temp']
    .mean()
)
```


In [23]:
monthly_temps = weather_df.groupby('month')['temp'].mean()
print("Average temperature by month:")
print(monthly_temps)

Average temperature by month:
month
1      7.373226
2      8.333451
3     10.649032
4     14.049167
5     17.093710
6     20.325667
7     22.283387
8     21.533871
9     19.163500
10    15.245806
11    10.780667
12     8.317581
Name: temp, dtype: float64


In [24]:
hot_days_by_month = weather_df.groupby('month')['is_hot'].sum()
print("\nTotal hot days by month:")
print(hot_days_by_month)


Total hot days by month:
month
1       0
2       0
3       0
4       2
5       8
6      66
7     126
8      73
9      25
10      2
11      0
12      0
Name: is_hot, dtype: int64


💭 **Personal Reflection:**

What patterns do you see in the monthly data? How might this help answer "Is London getting hotter?"

- [*Write your notes here*]


## Section 4: Presenting Your Data with pandas Styler

You now have summary tables from `.groupby()`. The next step is making them readable and presentable.

pandas Styler lets you **format**, **colour**, and **caption** DataFrames without changing the underlying data.


### The raw output problem

Look at `yearly_stats` again:

In [25]:
yearly_stats

,mean,max,min,std
year,,,,
2006,14.675342,31.5,0.8,6.764682
2007,14.390685,26.4,1.6,5.042399
2008,13.793443,26.6,2.0,5.497393
2009,13.921370,26.5,-0.1,6.040721
2010,12.749589,28.0,-1.5,7.180979
2011,14.774795,28.6,1.5,5.505723
2012,13.512295,28.1,-0.5,5.665366
2013,13.309589,30.4,-0.9,6.817386
2014,14.978356,28.5,4.0,5.285908


Too many decimal places. No visual hierarchy. Hard to scan for patterns.

Let's fix that.

### Styler basics: `.style.format()`


In [26]:
!pip install jinja2
yearly_stats.style.format("{:.1f}")


[notice] A new release of pip is available: 25.1.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


,mean,max,min,std
year,,,,
2006,14.7,31.5,0.8,6.8
2007,14.4,26.4,1.6,5.0
2008,13.8,26.6,2.0,5.5
2009,13.9,26.5,-0.1,6.0
2010,12.7,28.0,-1.5,7.2
2011,14.8,28.6,1.5,5.5
2012,13.5,28.1,-0.5,5.7
2013,13.3,30.4,-0.9,6.8
2014,15.0,28.5,4.0,5.3


Every cell now shows 1 decimal place. The underlying data is unchanged. This is **presentation only**.

### Column-specific formatting


In [27]:
yearly_stats.style.format({
    "mean": "{:.1f}°C",
    "max":  "{:.1f}°C",
    "min":  "{:.1f}°C",
    "std":  "{:.2f}"
})

,mean,max,min,std
year,,,,
2006,14.7°C,31.5°C,0.8°C,6.76
2007,14.4°C,26.4°C,1.6°C,5.04
2008,13.8°C,26.6°C,2.0°C,5.50
2009,13.9°C,26.5°C,-0.1°C,6.04
2010,12.7°C,28.0°C,-1.5°C,7.18
2011,14.8°C,28.6°C,1.5°C,5.51
2012,13.5°C,28.1°C,-0.5°C,5.67
2013,13.3°C,30.4°C,-0.9°C,6.82
2014,15.0°C,28.5°C,4.0°C,5.29


Different columns can have different precision and units. Temperature columns get °C, standard deviation keeps two decimal places.

### `.background_gradient()` for visual patterns


In [28]:
yearly_stats.style.background_gradient(subset=['mean'], cmap='YlOrRd')

,mean,max,min,std
year,,,,
2006,14.675342,31.500000,0.800000,6.764682
2007,14.390685,26.400000,1.600000,5.042399
2008,13.793443,26.600000,2.000000,5.497393
2009,13.921370,26.500000,-0.100000,6.040721
2010,12.749589,28.000000,-1.500000,7.180979
2011,14.774795,28.600000,1.500000,5.505723
2012,13.512295,28.100000,-0.500000,5.665366
2013,13.309589,30.400000,-0.900000,6.817386
2014,14.978356,28.500000,4.000000,5.285908


The colour gradient highlights values using a colour map. Hotter years get warmer colours.

Can you see the warming trend now? The colour does the work.

<div style="background-color: #fcfcfc; width:80%; margin-left: 1em; color: #212121; padding: 1em; border-radius: 0.5em; border-left: 5px solid #ff9800;">

**Key Idea:** `cmap` stands for "colour map". `'YlOrRd'` goes from yellow (low) to red (high). Other useful maps: `'Blues'`, `'RdYlGn'`, `'coolwarm'`.

</div>

### `.bar()` for inline comparison


In [29]:
yearly_stats.style.format(precision=2).bar(subset=['max'], color='#ED9255')

,mean,max,min,std
year,,,,
2006,14.68,31.50,0.80,6.76
2007,14.39,26.40,1.60,5.04
2008,13.79,26.60,2.00,5.50
2009,13.92,26.50,-0.10,6.04
2010,12.75,28.00,-1.50,7.18
2011,14.77,28.60,1.50,5.51
2012,13.51,28.10,-0.50,5.67
2013,13.31,30.40,-0.90,6.82
2014,14.98,28.50,4.00,5.29


Inline bars within cells give immediate visual comparison of magnitude. You can see which years had the highest maximum temperatures at a glance.

### Combining Styler methods

Chain methods together to build a complete presentation:

In [30]:
(
    yearly_stats.style
    .format({
        "mean": "{:.1f}°C",
        "max":  "{:.1f}°C",
        "min":  "{:.1f}°C",
        "std":  "{:.2f}"
    })
    .background_gradient(subset=['mean'], cmap='YlOrRd')
    .bar(subset=['max'], color='#ED9255')
    .set_caption("London's average temperature has risen 1.5°C over 20 years")
)

,mean,max,min,std
year,,,,
2006,14.7°C,31.5°C,0.8°C,6.76
2007,14.4°C,26.4°C,1.6°C,5.04
2008,13.8°C,26.6°C,2.0°C,5.50
2009,13.9°C,26.5°C,-0.1°C,6.04
2010,12.7°C,28.0°C,-1.5°C,7.18
2011,14.8°C,28.6°C,1.5°C,5.51
2012,13.5°C,28.1°C,-0.5°C,5.67
2013,13.3°C,30.4°C,-0.9°C,6.82
2014,15.0°C,28.5°C,4.0°C,5.29


`.set_caption()` is where your **narrative title** goes. The caption tells the reader what the table means, not what it contains.

## Live Exercise: The Compression Challenge

If we have time, we'll try this in class. Using `weather_df`, produce ONE styled table that answers:

**Which months have changed the most between 2006-2015 and 2016-2025?**

Constraints: maximum 6 rows, maximum 4 columns. You choose what to aggregate, which months to keep, and how to style it.


## Mini-Project 1 Support

You now have the tools to:

- Apply custom transformations with functions and `.apply()`
- Extract date components for temporal analysis
- Group data by time periods using `.groupby()`
- Present summary tables with formatting, gradients, and narrative captions

Continue experimenting with your OpenWeather API data in Mini-Project 1. The skills you learned today directly support NB02 (transformation) and NB03 (insights).

Tomorrow's lab will teach you matplotlib and seaborn for creating plots, the other presentation option for NB03.


---

📖 **Additional Resources:**
- [Pandas .apply() Method](https://pandas.pydata.org/docs/reference/api/pandas.Series.apply.html)
- [Pandas .groupby() Guide](https://pandas.pydata.org/docs/user_guide/groupby.html)
- [Pandas Styler Guide](https://pandas.pydata.org/docs/user_guide/style.html)
- [Matplotlib Datetime](https://matplotlib.org/stable/gallery/text_labels_and_annotations/date.html)
